In [2]:
import pandas as pd 
import numpy as np 

In [7]:
df  = pd.read_csv("./Factive_Scripts_Filenames - Sheet2.csv")
df

,Filename,Question,Condition,Setting,Correct_Answer
0,School_FB_1,Does Dr. C know where Billy is?,FB,School,N
1,School_FB_2,Does Jane know where Billy is?,FB,School,N
2,School_Alter_1,Does Dr. C know where Billy is?,Altercentric,School,N
3,School_Alter_2,Does Jane know where Billy is?,Altercentric,School,N
4,School_Ego_1,Does Dr. C know where Billy is?,Egocentric,School,Y
...,...,...,...,...,...
61,Spy_Filler_6,Are there two chairs in the command room?,Filler,Spy,N
62,Spy_Filler_7,Is the command room floor brown?,Filler,Spy,N
63,Spy_Filler_8,Are there stairs in the control room?,Filler,Spy,Y
64,Spy_Filler_9,Is there a dog in the command room?\n,Filler,Spy,N


In [20]:
''' 
Here, we will construct dictionaries for the filler.
Format: {question_text: [filename, correct_answer]}.
These dictionaries will be saved as js variables to be accessed in the main file. 
'''

pirate_filler = {}
spy_filler = {}
school_filler = {}
#school filler only has len 9 
for setting, group in df.groupby("Setting"):
    if setting == "Pirate":
        for question, group2 in group.groupby("Question"):
            if list(group2["Condition"])[0] == "Filler" or list(group2["Condition"])[0] == "Attention Check":
                filename = list(group2["Filename"])[0]
                correct_answer = list(group2["Correct_Answer"])[0]
                pirate_filler[question] = [filename, correct_answer ]
    if setting == "Spy":
        for question, group2 in group.groupby("Question"):
            if list(group2["Condition"])[0] == "Filler" or list(group2["Condition"])[0] == "Attention Check":
                filename = list(group2["Filename"])[0]
                correct_answer = list(group2["Correct_Answer"])[0]
                spy_filler[question] = [filename, correct_answer ]
    if setting == "School":
        for question, group2 in group.groupby("Question"):
            if list(group2["Condition"])[0] == "Filler" or list(group2["Condition"])[0] == "Attention Check":
                filename = list(group2["Filename"])[0]
                correct_answer = list(group2["Correct_Answer"])[0]
                school_filler[question] = [filename, correct_answer ]

{'Are Marias shoes green?': ['Pirate_Filler_9', 'Y'],
 'Are there palm trees?': ['Pirate_Filler_6', 'Y'],
 'Are there three hammocks?': ['Pirate_Filler_5', 'N'],
 'Does Maria have blonde hair? ': ['Pirate_Filler_10', 'N'],
 'Does the Captain have a beard? ': ['Pirate_Filler_2', 'Y'],
 'Does the tree house have 4 levels?': ['Pirate_Filler_7', 'N'],
 'Is it raining?': ['Pirate_Filler_4', 'N'],
 'Is the Captain’s uniform blue?': ['Pirate_Filler_1', 'Y'],
 'Is there a cannon?': ['Pirate_Filler_8', 'Y'],
 'Is there a pig roasting on the fire?': ['Pirate_Filler_3', 'N']}

In [36]:
school_filler

{'Are Dr. C’s shoes red?': ['School_Filler_7', 'N'],
 'Are the students sitting in a circle?': ['School_Filler_10', 'N'],
 "Are the walls in Dr. C's classroom yellow?": ['School_Filler_3', 'N'],
 'Are there curtains in the classroom window?': ['School_Filler_6', 'N'],
 'Are there two tables in the history classroom?': ['School_Filler_1', 'Y'],
 'Is Janes shirt yellow?': ['School_Filler_4', 'N'],
 "Is there a globe in Dr. C's classroom?": ['School_Filler_9', 'Y'],
 "Is there a plant in Dr. C's classroom?": ['School_Filler_2', 'Y'],
 "Is there a timeline in Dr. C's classroom?": ['School_Filler_8', 'Y']}

In [33]:
'''
Example for how target question dic should be made. 
condition_qs = {
    "school":{
        "FB": {"Does Dr. C know where Billy is?":"N", "Does Jane know where Billy is?":"N"}, 
        "SI":{"Does Dr. C know where Billy is?":"N", "Does Jane know where Billy is?":"N"}, 
        "EI":{"Does Dr. C know where Billy is?":"Y", "Does Jane know where Billy is?":"N"}, 
        "NI":{"Does Dr. C know where Billy is?":"N", "Does Jane know where Billy is?":"N"}, 
        "TB":{"Does Dr. C know where Billy is?":"Y", "Does Jane know where Billy is?":"N"}, 
        "G":{"Does Dr. C know where Billy is?":"Y", "Does Jane know where Billy is?":"Y"}, 
    }
} 
Format: {setting: {condition: {target_question: [filename, correct_answer]}}
}
'''
conversion = {"Altercentric": "AI", 
            "Egocentric": "EI",
            "FB":"FB", 
            "Gettier": "G",
            "Non-specific":"NI", 
            "TB":"TB",
}
target_questions = {}
for setting, group in df.groupby("Setting"):
    target_questions[setting] = {}
    for  condition, group2 in group.groupby("Condition"):
        condition = condition.strip(" ")
        if condition != "Filler" and condition != "Attention Check":
            condition = conversion[condition]
            target_questions[setting][condition] = {}
            for question, group3 in group2.groupby("Question"):
                filename = list(group3["Filename"])[0]
                correct_answer = list(group3["Correct_Answer"])[0]
                target_questions[setting][condition][question] = [filename, correct_answer]
            
target_questions

{'Pirate': {'AI': {'Does Harry know where Maria is?': ['Pirate_Alter_2', 'N'],
   'Does the Captain know where Maria is?': ['Pirate_Alter_1', 'N']},
  'EI': {'Does Harry know where Maria is?': ['Pirate_Ego_2', 'N'],
   'Does the Captain know where Maria is?': ['Pirate_Ego_1', 'Y']},
  'FB': {'Does Harry know where Maria is?': ['Pirate_FB_2', 'N'],
   'Does the Captain know where Maria is?': ['Pirate_FB_1', 'N']},
  'G': {'Does Harry know where Maria is?': ['Pirate_Gettier_2', 'Y'],
   'Does the Captain know where Maria is?': ['Pirate_Gettier_1', 'Y']},
  'NI': {'Does Harry know where Maria is?': ['Pirate_NS_2', 'N'],
   'Does the Captain know where Maria is? ': ['Pirate_NS_1', 'N']},
  'TB': {'Does Harry know where Maria is?': ['Pirate_TB_2', 'N'],
   'Does the Captain know where Maria is?': ['Pirate_TB_1', 'Y']}},
 'School': {'AI': {'Does Dr. C know where Billy is?': ['School_Alter_1', 'N'],
   'Does Jane know where Billy is?': ['School_Alter_2', 'N']},
  'EI': {'Does Dr. C know where